In [0]:
#importando bibliotecas
import boto3  # para conexão com aws
import json   # para organizar ler e salvar json 
import requests  #para uso de solicitações da API





In [0]:
# função recuperar dados da api
def extract_all_pokemon():
  base_url = "https://pokeapi.co/api/v2/pokemon/"
  pokemon_list = []
  next_url = base_url

  while next_url:
    response = requests.get(next_url)
    if response.status_code == 200:
      data = response.json()
      pokemon_list.extend(data['results'])
      next_url = data.get('next')
    else:
      raise Exception(f"Failed to fetch pokémon data. Satatus code: {response.status_code}")
  
  return pokemon_list

In [0]:
# recebenco os valores da função e gravando em um dataframe
pokemons = spark.createDataFrame(extract_all_pokemon())


In [0]:
# função para salvar dados no S3
def save_pokemon_to_s3(json_data, bucket_name, file_name):
    try:
        s3 = boto3.client('s3')
        json_string = json.dumps(json_data, indent=4)
        s3_client.put_object(Bucket=bucket_name, Key=file_name, Body=json_string)

        print(f"Arquivo {file_name} Salvo com sucesso no buckeet {bucket_name}")
    except Exception as e:
        print(f"Erro ao salvar arquivo no S3: {str(e)}")

In [0]:
# mostrando o dataframe
display(pokemons)

In [0]:
# criando a estrutura e salvando o dataframe em uma table
spark.sql("CREATE CATALOG IF NOT EXISTS pokemon")
spark.sql("CREATE SCHEMA IF NOT EXISTS pokemon.bronze")
# salvando no bronze
pokemons.write.mode("overwrite").saveAsTable("pokemon.bronze.API")

In [0]:
%sql
-- Selecionando as informações da tabela pokemon.bronze.API
SELECT * FROM pokemon.bronze.api